# Tools and Routing

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.agents import tool

/Users/christy/Documents/py312/lib/python3.12/site-packages/langchain_community/utilities/__init__.py:8: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.utilities.requests import (


In [3]:
# LangChain tag makes function like Pydantic class?
# args includes default title: value only
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"


In [ ]:
search.name

In [ ]:
search.description

In [ ]:
search.args

# {'query': {'description': 'Thing to search for',
#   'title': 'Query',
#   'type': 'string'}}

In [ ]:
search.run()

In [16]:
# Recommended pattern: Pydantic + langChain @tools decorator
# Clearer function's expected input.
from pydantic import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"


In [ ]:
search.args

In [ ]:
# Doesnt do anything!
search.run("dc")

In [17]:
import requests, datetime
from pydantic import BaseModel, Field

# Pydantic: define the input args schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

# LangChain tools decorator, adds metadata
# - Converts Pydantic class to OpenAI function definition.
# - Input args runtime will be type checked against expected param schema.
# - Function can be called by a LLM, which dynamically decides 
#   whether or not to call this function and params.
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    # API endpoint
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Actual API call, as a http get request.
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    # Process the response to find the current temperature
    current_utc_time = datetime.datetime.now()
    # Get list of times from response results.
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    # Get list of temps from response results.
    temperature_list = results['hourly']['temperature_2m']
    # Get index of closest time.
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    # Lookup corresponding temp at that index.
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [ ]:
get_current_temperature.name

In [ ]:
get_current_temperature.description

In [ ]:
get_current_temperature.args

In [18]:
from langchain.tools.render import format_tool_to_openai_function

In [19]:
# Inspect the OpenAI JSON blob.
format_tool_to_openai_function(get_current_temperature)

/var/folders/cx/j9s41z_97kd3g0jh0qf987m00000gn/T/ipykernel_62068/2884313990.py:2: LangChainDeprecationWarning: The function `_format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  format_tool_to_openai_function(get_current_temperature)


{'name': 'get_current_temperature',
 'description': 'Fetch current temperature for given coordinates.',
 'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [ ]:
# Costs money for OpenAI call
get_current_temperature({"latitude": 13, "longitude": 14})

In [25]:
# Costs money for OpenAI call
result = get_current_temperature({"latitude": 13, "longitude": 14})

In [ ]:
print(type(result))
result

# Real call to functions

In [ ]:
!python3 -m pip install wikipedia

In [20]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []

    # Max return summaries of only first 3 wikipedia pages.
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    
    # Return single string of summaries
    return "\n\n".join(summaries)

In [ ]:
search_wikipedia.name

In [ ]:
search_wikipedia.description

In [ ]:
format_tool_to_openai_function(search_wikipedia)

In [ ]:
# This costs $
import pprint
results = search_wikipedia({"query": "langchain"})
pprint.pprint(result)

# 3 pages found!

In [ ]:
type(result)

# Function calling using OpenAI specification

Given a list of 3 functions, test LLM can call the right <br>
function with the right params.

In [ ]:
# Modify pip install and imports according to:
# https://community.deeplearning.ai/t/attributeerror-super-object-has-no-attribute-parse-obj/481376/12

# !pip install --upgrade pip
# !python3 -m pip install -U langchain  # '0.3.19'
# !python3 -m pip install -U openapi-pydantic openapi-schema-pydantic

In [4]:
# from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
# from langchain.utilities.openapi import OpenAPISpec

from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec
from openapi_pydantic import OpenAPI, Info, PathItem, Operation, Response


In [6]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [7]:
spec = OpenAPISpec.from_text(text)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [8]:
# Convert these JSON function definitions to openAI spec.
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [9]:
pet_openai_functions

# We have 3 functions: listPets, createPets, showPetById

[{'name': 'listPets',
  'description': 'List all pets',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'limit': {'type': 'integer',
       'maximum': 100.0,
       'schema_format': 'int32',
       'description': 'How many items to return at one time (max 100)'}},
     'required': []}}}},
 {'name': 'createPets',
  'description': 'Create a pet',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'showPetById',
  'description': 'Info for a specific pet',
  'parameters': {'type': 'object',
   'properties': {'path_params': {'type': 'object',
     'properties': {'petId': {'type': 'string',
       'description': 'The id of the pet to retrieve'}},
     'required': ['petId']}}}}]

In [ ]:
from langchain_openai import ChatOpenAI

In [12]:
# Instantiate model, bind openai functions from spec.
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

In [13]:
# Test what function gets called + their params.
model.invoke("what are three pets names")

# 'function_call': {'arguments': '{"params":{"limit":3}}', 'name': 'listPets'}

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"params":{"limit":3}}', 'name': 'listPets'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 123, 'total_tokens': 140, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-eac45d0d-cdeb-434a-a152-11cee2ccb00f-0', usage_metadata={'input_tokens': 123, 'output_tokens': 17, 'total_tokens': 140})

In [14]:
# Test what function gets called + their params.
model.invoke("tell me about pet with id 42")

# 'function_call': {'arguments': '{"path_params":{"petId":"42"}}', 'name': 'showPetById'}

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"path_params":{"petId":"42"}}', 'name': 'showPetById'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 126, 'total_tokens': 146, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-aae9a4fd-4dd1-4bd4-a67a-c733618bf142-0', usage_metadata={'input_tokens': 126, 'output_tokens': 20, 'total_tokens': 146})

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [21]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [22]:
model.invoke("what is the weather in dc right now")

# 'function_call': {'arguments': '{"latitude":38.9072,"longitude":-77.0369}', 'name': 'get_current_temperature'}

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":38.9072,"longitude":-77.0369}', 'name': 'get_current_temperature'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 105, 'total_tokens': 131, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-6791498d-04e0-4d62-ac85-713af543815c-0', usage_metadata={'input_tokens': 105, 'output_tokens': 26, 'total_tokens': 131})

In [23]:
result = model.invoke("what is ag2 of autogen")

# 'function_call': {'arguments': '{"query":"Ag2"}', 'name': 'search_wikipedia'}

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Ag2"}', 'name': 'search_wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 101, 'total_tokens': 118, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-4ebb6993-ead3-40ed-8232-3984ef706a00-0', usage_metadata={'input_tokens': 101, 'output_tokens': 17, 'total_tokens': 118})

In [24]:
# Same thing, but use a chained prompt to model.
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [ ]:
chain.invoke({"input": "what is the weather in sf right now"})

In [51]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [52]:
# Same thing, but chained prompt to model to parser.
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [63]:
result = chain.invoke({"input": "what is ag2 library autogen?"})

In [64]:
type(result)

# langchain_core.agents.AgentActionMessageLog
# This means a tool will be called!

langchain_core.agents.AgentActionMessageLog

In [65]:
result.tool

'search_wikipedia'

In [66]:
result.tool_input

{'query': 'ag2 library autogen'}

In [67]:
# get_current_temperature(result.tool_input)
results = search_wikipedia(result.tool_input)

In [68]:
import pprint
pprint.pprint(results)

'No good Wikipedia Search Result was found'


In [69]:
result = chain.invoke({"input": "hi!"})

In [70]:
type(result)

# type(result)

# langchain_core.agents.AgentFinish
# No function calling :(

langchain_core.agents.AgentFinish

In [71]:
# Since result was type AgentFinish, check .return_values
result.return_values

{'output': 'Well, hello there! How can I assist you today?'}

# Putting it all together:
- Define openai spec functions
- Define a Router that checks type of result
- If type AgentFinish, then get function name, params
  - invoke the function.run(params)

In [72]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [73]:
# Chain: prompt to model to parser to Router
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [74]:
results = chain.invoke({"input": "What is the weather in dc right now?"})

In [75]:
results

'The current temperature is 9.2°C'

In [76]:
results = chain.invoke({"input": "What is autogen agents package?"})

In [77]:
results

'No good Wikipedia Search Result was found'

In [78]:
chain.invoke({"input": "hi!"})

'Well, hello there! How can I assist you today?'